In [2]:
import pandas as pd
import numpy as np

data = {
    'size': ['M','S',np.nan,'M','XL'],
    'color': ['green', 'blue', 'blue', np.nan, np.nan],
    'price': [200, np.nan, 200, 300, 300],
    'quantity': [999, np.nan, 10000, np.nan, 10000]
}
X = pd.DataFrame(data)
X_orig = X.copy()
X.style.highlight_null(color='yellow')

,size,color,price,quantity
0,M,green,200.000000,999.000000
1,S,blue,nan,nan
2,nan,blue,200.000000,10000.000000
3,M,nan,300.000000,nan
4,XL,nan,300.000000,10000.000000


In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   size      4 non-null      object 
 1   color     3 non-null      object 
 2   price     4 non-null      float64
 3   quantity  3 non-null      float64
dtypes: float64(2), object(2)
memory usage: 292.0+ bytes


In [5]:
X_col_num = ['price','quantity']
X_num = X[X_col_num]
X_num.style.highlight_null(color='yellow')

,price,quantity
0,200.000000,999.000000
1,nan,nan
2,200.000000,10000.000000
3,300.000000,nan
4,300.000000,10000.000000


In [8]:
from sklearn.impute import SimpleImputer

# 這段程式碼使用了 sklearn.impute.SimpleImputer 這個類別，它是用來處理缺失值的 。在這段程式碼中，strategy='mean' 表示使用平均值來填補缺失值 

si = SimpleImputer(strategy='mean')
X_num_impute = si.fit_transform(X_num)
X_num_impute

array([[  200.        ,   999.        ],
       [  250.        ,  6999.66666667],
       [  200.        , 10000.        ],
       [  300.        ,  6999.66666667],
       [  300.        , 10000.        ]])

In [9]:
# si.statistics_ 是 SimpleImputer 類別的一個屬性，它會回傳每個欄位的統計值 。在這段程式碼中，si 是一個 SimpleImputer 的物件，而 statistics_ 則是這個物件的一個屬性，它會回傳填補過缺失值的結果 。

si.statistics_

array([ 250.        , 6999.66666667])

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
ss = StandardScaler()
X_num_impute_ss = ss.fit_transform(X_num_impute)
X_num_impute_ss

# 這是一段 Python 程式碼，使用了 scikit-learn 套件中的 StandardScaler() 和 MinMaxScaler() 類別。
# StandardScaler() 類別可以將資料進行標準化，使得每個特徵的平均值為 0，標準差為 1。而 MinMaxScaler() 類別則可以將資料進行縮放，
# 使得每個特徵的數值都在 0 到 1 之間。在這段程式碼中，StandardScaler() 類別被用來對 X_num_impute 資料進行標準化，
# 而 MinMaxScaler() 類別則被用來對標準化後的資料進行縮放。

array([[-1.11803399, -1.82574186],
       [ 0.        ,  0.        ],
       [-1.11803399,  0.91287093],
       [ 1.11803399,  0.        ],
       [ 1.11803399,  0.91287093]])

In [11]:
df_X_num_impute = pd.DataFrame(X_num_impute)
(df_X_num_impute - df_X_num_impute.mean())/df_X_num_impute.std(ddof=0)

# 用於對 DataFrame 進行標準化。
# 首先將 X_num_impute 資料轉換為 DataFrame 格式，並命名為 df_X_num_impute。接著，
# 使用 df_X_num_impute.mean() 函數計算 df_X_num_impute 的平均值，
# 再使用 df_X_num_impute.std(ddof=0) 函數計算 df_X_num_impute 的標準差。
# 最後，使用 (df_X_num_impute - df_X_num_impute.mean())/df_X_num_impute.std(ddof=0) 對 df_X_num_impute 資料進行標準化

,0,1
0,-1.118034,-1.825742
1,0.000000,0.000000
2,-1.118034,0.912871
3,1.118034,0.000000
4,1.118034,0.912871


In [12]:
from sklearn.pipeline import make_pipeline
num_pl = make_pipeline(SimpleImputer(strategy='mean'),
                       StandardScaler())
num_pl.fit_transform(X_num)

# 使用了 scikit-learn 套件中的 make_pipeline() 函數和 SimpleImputer() 類別以及 StandardScaler() 類別。make_pipeline() 函數可以將多個轉換器組合成一個管道，
# 使得資料可以依序經過多個轉換器進行處理。在這段程式碼中，SimpleImputer() 類別被用來對缺失值進行填補，而 StandardScaler() 類別則被用來對資料進行標準化。
# 最後，使用 num_pl.fit_transform(X_num) 對 X_num 資料進行填補和標準化的處理

array([[-1.11803399, -1.82574186],
       [ 0.        ,  0.        ],
       [-1.11803399,  0.91287093],
       [ 1.11803399,  0.        ],
       [ 1.11803399,  0.91287093]])

In [13]:
X_col_cat = ['size','color']
X_cat = X[X_col_cat]
X_cat.style.highlight_null(color='yellow')

,size,color
0,M,green
1,S,blue
2,nan,blue
3,M,nan
4,XL,nan


In [14]:
si = SimpleImputer(strategy='most_frequent')
X_cat_impute = si.fit_transform(X_cat)
X_cat_impute

# 使用了 scikit-learn 套件中的 SimpleImputer() 類別。SimpleImputer() 類別可以將缺失值進行填補，
# 填補的方式可以是平均值、中位數、眾數或常數。在這段程式碼中，strategy='most_frequent' 表示使用眾數來填補缺失值。
# si.fit_transform(X_cat) 則是對 X_cat 資料進行填補的處理

array([['M', 'green'],
       ['S', 'blue'],
       ['M', 'blue'],
       ['M', 'blue'],
       ['XL', 'blue']], dtype=object)

In [15]:
pd.get_dummies(X_cat)

,size_M,size_S,size_XL,color_blue,color_green
0,True,False,False,False,True
1,False,True,False,True,False
2,False,False,False,True,False
3,True,False,False,False,False
4,False,False,True,False,False


In [16]:
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder(sparse=False)
X_cat_impute_oh = oh.fit_transform(X_cat_impute)
X_cat_impute_oh

# 使用了 scikit-learn 套件中的 OneHotEncoder() 類別。OneHotEncoder() 類別可以將類別型特徵進行編碼，
# 將每個類別轉換成一個二元特徵，其中一個特徵表示該樣本是否屬於該類別，另一個特徵則表示該樣本是否不屬於該類別。
# 在這段程式碼中，sparse=False 表示輸出的編碼結果是一個二維的 Numpy 陣列，而不是一個稀疏矩陣。
# oh.fit_transform(X_cat_impute) 則是對 X_cat_impute 資料進行編碼的處理

c:\Users\11706\.conda\envs\TestEnv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 1., 1., 0.]])

In [17]:
oh.get_feature_names_out(['size','color'])

array(['size_M', 'size_S', 'size_XL', 'color_blue', 'color_green'],
      dtype=object)

In [19]:
cat_pl = make_pipeline(SimpleImputer(strategy='most_frequent'),
                       OneHotEncoder(sparse=False))
cat_pl.fit_transform(X_cat)

# 這段程式碼使用了 scikit-learn 的 Pipeline 類別，它可以將多個轉換器和估計器（比如 SimpleImputer 和 OneHotEncoder）按照順序連接起來，
# 形成一個機器學習管道。在這個例子中，首先使用 SimpleImputer 將缺失值以最常見的值填補，然後使用 OneHotEncoder 將類別變數轉換為數值變數，
# 最後使用 fit_transform 方法將轉換器應用到輸入資料集 X_cat 上，並返回轉換後的資料集。

c:\Users\11706\.conda\envs\TestEnv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 1., 1., 0.]])

In [20]:
oh_in_pl = cat_pl.named_steps['onehotencoder']
oh_in_pl.get_feature_names_out(['size','color'])

# 這段程式碼使用了 scikit-learn 的 Pipeline 類別，
# 它可以將多個轉換器和估計器（比如 SimpleImputer 和 OneHotEncoder）按照順序連接起來，形成一個機器學習管道。
# 在這個例子中，首先使用 SimpleImputer 將缺失值以最常見的值填補，然後使用 OneHotEncoder 將類別變數轉換為數值變數，
# 最後使用 fit_transform 方法將轉換器應用到輸入資料集 X_cat 上，並返回轉換後的資料集。

array(['size_M', 'size_S', 'size_XL', 'color_blue', 'color_green'],
      dtype=object)

In [21]:
np.concatenate([X_num_impute_ss, X_cat_impute_oh], axis=1).round(2)

# 這段程式碼使用了 numpy.concatenate 函數，它可以將多個 numpy 陣列沿著指定的軸進行連接。
# 在這裡，我們使用 axis=1 將 X_num_impute_ss 和 X_cat_impute_oh 沿著第二個軸進行連接。
# 最後，我們使用 round(2) 函數將結果四捨五入到小數點後兩位。

array([[-1.12, -1.83,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ],
       [-1.12,  0.91,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ],
       [ 1.12,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ],
       [ 1.12,  0.91,  0.  ,  0.  ,  1.  ,  1.  ,  0.  ]])

In [22]:
from sklearn.compose import ColumnTransformer
data_pl = ColumnTransformer([
    ('num_pl', num_pl, X_col_num),
    ('cat_pl', cat_pl, X_col_cat)
])
data_pl.fit_transform(X).round(2)

# 從你傳的程式碼中，我們可以看到你使用了 ColumnTransformer 類別，它可以將不同的預處理步驟應用於資料集的不同欄位。
# 在這段程式碼中，我們建立了一個名為 data_pl 的 ColumnTransformer 實例，它有兩個轉換器：num_pl 和 cat_pl，
# 分別應用於數值和類別欄位。數值欄位由 X_col_num 指定，而類別欄位由 X_col_cat 指定。
# 最後，我們呼叫 data_pl 的 fit_transform() 方法來預處理資料，並使用 round(2) 函式將結果四捨五入到小數點後兩位

c:\Users\11706\.conda\envs\TestEnv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[-1.12, -1.83,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ],
       [-1.12,  0.91,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ],
       [ 1.12,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ],
       [ 1.12,  0.91,  0.  ,  0.  ,  1.  ,  1.  ,  0.  ]])

In [23]:
from sklearn.compose import ColumnTransformer
data_pl = ColumnTransformer([
    ('num_pl', 'drop', X_col_num),
    ('cat_pl', cat_pl, X_col_cat)
])
pd.DataFrame(data_pl.fit_transform(X))

# 這段程式碼使用了 ColumnTransformer 類別，它可以將不同的預處理步驟應用於資料集的不同欄位。
# 在這段程式碼中，我們建立了一個名為 data_pl 的 ColumnTransformer 實例，它有兩個轉換器：num_pl 和 cat_pl，分別應用於數值和類別欄位。
# 數值欄位由 X_col_num 指定，而類別欄位由 X_col_cat 指定。
# 最後，我們呼叫 data_pl 的 fit_transform() 方法來預處理資料，並使用 pd.DataFrame() 函式將結果轉換成 pandas.DataFrame 格式

c:\Users\11706\.conda\envs\TestEnv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4
0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,1.0,0.0
